# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
geoapify_key = "6d15b6bb7a8a4c2baff806e956d5ebfd"

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv('../output_data/cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,muisne,0.6000,-80.0333,24.91,90,31,1.98,EC,1675828567
1,1,port elizabeth,-33.9180,25.5701,19.62,90,0,3.60,ZA,1675828568
2,2,rosetta,31.4044,30.4164,12.40,65,100,8.40,EG,1675828568
3,3,broome,42.2506,-75.8330,3.90,81,100,5.14,US,1675828569
4,4,bull savanna,17.8868,-77.5902,21.39,82,0,3.15,JM,1675828571


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
#%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    tiles = 'OSM',
    geo = True,
    frame_width= 700,
    frame_height= 500,
    size= 'Humidity',
    scale= 0.5,
    color= 'City')

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df['Max Temp'] < 27.00) & (city_data_df['Max Temp'] > 21.00)
    & (city_data_df['Wind Speed'] < 4.5)
    & (city_data_df['Cloudiness'] == 0)
,:]
# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,bull savanna,17.8868,-77.5902,21.39,82,0,3.15,JM,1675828571
185,185,black river,18.0264,-77.8487,21.93,84,0,2.37,JM,1675828668
227,227,niamey,13.5137,2.1098,24.16,15,0,2.57,NE,1675828683
343,343,pedernales,18.0384,-71.7440,24.79,64,0,2.16,DO,1675828480
419,419,itaqui,-29.1253,-56.5531,25.31,25,0,4.08,BR,1675828746
472,472,lashio,22.9333,97.7500,22.64,28,0,0.94,MM,1675828763
512,512,hesla,24.0667,85.8833,23.70,32,0,2.66,IN,1675828776
526,526,raver,21.2500,76.0333,24.51,21,0,3.85,IN,1675828781
539,539,senador canedo,-16.7081,-49.0931,22.63,83,0,0.00,BR,1675828785


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Max Temp", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
4,bull savanna,JM,21.39,17.8868,-77.5902,82,
185,black river,JM,21.93,18.0264,-77.8487,84,
227,niamey,NE,24.16,13.5137,2.1098,15,
343,pedernales,DO,24.79,18.0384,-71.7440,64,
419,itaqui,BR,25.31,-29.1253,-56.5531,25,
472,lashio,MM,22.64,22.9333,97.7500,28,
512,hesla,IN,23.70,24.0667,85.8833,32,
526,raver,IN,24.51,21.2500,76.0333,21,
539,senador canedo,BR,22.63,-16.7081,-49.0931,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'api_key': geoapify_key
}
# Print a message to follow up the hotel search
print('Starting hotel search')
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # Add filter and bias parameters with the current city’s latitude and longitude to the params dictionary
    params['filter'] = f'circle:{lng},{lat},{radius}'
    params['bias'] = f'proximity:{lng},{lat}'
    # Set base URL
    base_url = 'https://api.geoapify.com/v2/places'
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as “No hotel found”.
        hotel_df.loc[index, 'Hotel Name'] = 'No hotel found'
    # Log the search results
    print(f"{hotel_df.loc[index,'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
bull savanna - nearest hotel: Junction Guest House
black river - nearest hotel: No hotel found
niamey - nearest hotel: Redidence Croix du Sud
pedernales - nearest hotel: Hostal Doña Chava
itaqui - nearest hotel: Hotel Contursi
lashio - nearest hotel: Golden Kaniri Hotel
hesla - nearest hotel: No hotel found
raver - nearest hotel: No hotel found
senador canedo - nearest hotel: No hotel found


,City,Country,Max Temp,Lat,Lng,Humidity,Hotel Name
4,bull savanna,JM,21.39,17.8868,-77.5902,82,Junction Guest House
185,black river,JM,21.93,18.0264,-77.8487,84,No hotel found
227,niamey,NE,24.16,13.5137,2.1098,15,Redidence Croix du Sud
343,pedernales,DO,24.79,18.0384,-71.7440,64,Hostal Doña Chava
419,itaqui,BR,25.31,-29.1253,-56.5531,25,Hotel Contursi
472,lashio,MM,22.64,22.9333,97.7500,28,Golden Kaniri Hotel
512,hesla,IN,23.70,24.0667,85.8833,32,No hotel found
526,raver,IN,24.51,21.2500,76.0333,21,No hotel found
539,senador canedo,BR,22.63,-16.7081,-49.0931,83,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
# %%capture --no-display

# Configure the map plot
hotel_name_on_map = hotel_df.hvplot.points(
  "Lng",
  "Lat",
  geo = True,
  tiles = "OSM",
  frame_width = 800,
  frame_height = 600,
  size = "Humidity",
  scale = 1,
  color = "City",
)
# Configure the map plot
hotel_name_on_map = hotel_df.hvplot.points(
  "Lng",
  "Lat",
  geo = True,
  tiles = "OSM",
  frame_width = 800,
  frame_height = 600,
  size = "Humidity",
  scale = 1,
  color = "City",
  hover_cols = ["Hotel Name","Country"]
)

# Display the map
hotel_name_on_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)